In [1]:
import genvarloader as gvl
import numba as nb
import numpy as np
import polars as pl
import seqpro as sp
import pooch

# GenVarLoader Tutorial: Geuvadis

In this tutorial we'll see how to use GenVarLoader (GVL) to:
1. Write a GVL dataset to disk
2. Inspect the dataset
3. Optional: write transformed versions of the tracks to disk
4. Add on-the-fly transformations
5. Obtain splits from the dataset
6. Get a PyTorch DataLoader

## Downloading the data

The Geuvadis dataset is 451 individuals from the 1000 Genomes Project that have both whole genome sequencing and RNA-seq from blood samples. We'll see how to use GVL to get a high performance dataloader that yields haplotypes and tracks for training or running inference with sequence models. For the sake of this tutorial, we'll only work with chromosome 22 so everything can run in a few minutes.

In [2]:
# GRCh38 chromosome 22 sequence
reference = pooch.retrieve(
    url="https://ftp.ensembl.org/pub/release-112/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz",
    known_hash="sha256:974f97ac8ef7ffae971b63b47608feda327403be40c27e391ee4a1a78b800df5",
    progressbar=True,
)
!gzip -dc {reference} | bgzip > {reference[:-3]}.bgz
reference = reference[:-3] + ".bgz"

# PLINK 2 files
variants = pooch.retrieve(
    url="doi:10.5281/zenodo.13656224/1kGP.chr22.pgen",
    known_hash="md5:31aba970e35f816701b2b99118dfc2aa",
    progressbar=True,
    fname="1kGP.chr22.pgen",
)
pooch.retrieve(
    url="doi:10.5281/zenodo.13656224/1kGP.chr22.psam",
    known_hash="md5:eefa7aad5acffe62bf41df0a4600129c",
    progressbar=True,
    fname="1kGP.chr22.psam",
)
pooch.retrieve(
    url="doi:10.5281/zenodo.13656224/1kGP.chr22.pvar",
    known_hash="md5:5f922af91c1a2f6822e2f1bb4469d12b",
    progressbar=True,
    fname="1kGP.chr22.pvar",
)

# BigWigs and sample ID mapping
bw_paths = pooch.retrieve(
    url="doi:10.5281/zenodo.13656224/bw_chr22.tar.gz",
    known_hash="md5:14bf72e9e9d3e2318d07315c4a2675fb",
    progressbar=True,
    processor=pooch.Untar(),
)
bw_table_path = pooch.retrieve(
    url="doi:10.5281/zenodo.13656224/bigwig_table.csv",
    known_hash="md5:7fe7c55b61c7dfa66cfd0a49336f3b08",
    progressbar=True,
)

# BED
bed_path = pooch.retrieve(
    url="doi:10.5281/zenodo.13656224/chr22_egenes.bed",
    known_hash="md5:ccb55548e4ddd416d50dbe6638459421",
    progressbar=True,
)

100%|█████████████████████████████████████| 11.4M/11.4M [00:00<00:00, 13.7GB/s]
100%|█████████████████████████████████████| 20.4M/20.4M [00:00<00:00, 26.3GB/s]
100%|█████████████████████████████████████| 6.38k/6.38k [00:00<00:00, 9.92MB/s]
100%|█████████████████████████████████████| 1.17G/1.17G [00:00<00:00, 1.35TB/s]
100%|█████████████████████████████████████| 1.01G/1.01G [00:00<00:00, 1.30TB/s]
Untarring contents of '/carter/users/dlaub/.cache/pooch/c91b5d0155f29d91dddf23901e396774-bw_chr22.tar.gz' to '/carter/users/dlaub/.cache/pooch/c91b5d0155f29d91dddf23901e396774-bw_chr22.tar.gz.untar'
0.00B [00:00, ?B/s]
100%|█████████████████████████████████████| 7.26k/7.26k [00:00<00:00, 8.30MB/s]


## Writing the GVL dataset

We'll specify a path to store the dataset, which is a directory (like Zarr stores if you're familiar with those).

In [3]:
ds_path = "geuvadis.chr22.gvl"

We'll also need a table or dictionary specifying the sample names for each BigWig. We'll use a table here, which must have at least have columns `sample` and `path` as seen below. The join is added here to update the paths to match the actual download paths.

In [4]:
bigwig_table = (
    pl.read_csv(bw_table_path)
    .join(
        pl.Series(bw_paths).to_frame("realpath"),
        left_on="path",
        right_on=pl.col("realpath").str.split("/").list.get(-1),
    )
    .drop("path")
    .rename({"realpath": "path"})
)
bigwig_table.head()

sample,read_count,path
str,i64,str
"""HG00285""",26025190,"""/carter/users/dlaub/.cache/poo…"
"""NA20514""",26304702,"""/carter/users/dlaub/.cache/poo…"
"""NA19152""",27759874,"""/carter/users/dlaub/.cache/poo…"
"""NA20581""",23282559,"""/carter/users/dlaub/.cache/poo…"
"""NA18910""",17936970,"""/carter/users/dlaub/.cache/poo…"


Finally, we'll need a BED file specifying what regions to include in the dataset. We can either specify a path or a polars DataFrame. We'll use `gvl.read_bedlike` to conveniently read the BED file into memory and subset it to just the first 5 regions for this tutorial. The BED file provided corresponds to eGenes, sorted in descending order by their absolute sum of coefficients.

In [5]:
bed = gvl.read_bedlike(bed_path)[:5]
bed.head()

chrom,chromStart,chromEnd,name,score,strand
str,i64,i64,str,f64,str
"""chr22""",41699499,41699499,"""ENSG00000167077""",null,"""+"""
"""chr22""",42835412,42835412,"""ENSG00000100266""",null,"""-"""
"""chr22""",20858983,20858983,"""ENSG00000099940""",null,"""+"""
"""chr22""",20707691,20707691,"""ENSG00000241973""",null,"""-"""
"""chr22""",49918167,49918167,"""ENSG00000184164""",null,"""+"""


Now, we're ready to write the dataset. We'll instantiate a gvl.BigWigs object here, which has alternative constructors in case we didn't want to use a table. We also name this track as "depth" (as in read depth) so we can manage different transformations of the track data or provide multiple tracks for the same samples. Later, we'll add a transformed track for $\log_2(\text{CPM}+1)$ to see this in action.

In [6]:
gvl.write(
    path=ds_path,
    bed=bed,
    variants=variants,
    bigwigs=gvl.BigWigs.from_table(name="depth", table=bigwig_table),
    length=2**15,
    max_jitter=128,
    overwrite=True,
)

2024-09-03 15:10:38.657 | INFO     | genvarloader._dataset._write:write:74 - Writing dataset to geuvadis.chr22.gvl
2024-09-03 15:10:38.659 | INFO     | genvarloader._dataset._write:write:79 - Found existing GVL store, overwriting.
2024-09-03 15:10:38.846 | INFO     | genvarloader._variants._records:read_pvar:432 - Reading .pvar file...
2024-09-03 15:10:40.321 | INFO     | genvarloader._variants._records:read_pvar:440 - Finished reading .pvar file.
2024-09-03 15:10:41.816 | INFO     | genvarloader._dataset._write:write:137 - Using 451 samples.
2024-09-03 15:10:41.818 | INFO     | genvarloader._dataset._write:write:143 - Writing genotypes.


  0%|          | 0/1 [00:00<?, ?it/s]

2024-09-03 15:10:42.083 | DEBUG    | genvarloader._dataset._write:_read_variants_chunk:381 - region length 34024
2024-09-03 15:10:42.084 | DEBUG    | genvarloader._dataset._write:_read_variants_chunk:387 - read genotypes
2024-09-03 15:10:50.604 | DEBUG    | genvarloader._dataset._write:_read_variants_chunk:398 - get haplotype region ilens
2024-09-03 15:10:50.648 | DEBUG    | genvarloader._dataset._write:_read_variants_chunk:404 - average haplotype length 34010.851219512195
2024-09-03 15:10:50.650 | DEBUG    | genvarloader._dataset._write:_read_variants_chunk:407 - max missing length -726
2024-09-03 15:10:50.651 | DEBUG    | genvarloader._dataset._write:_read_variants_chunk:417 - sparsify genotypes
2024-09-03 15:10:50.717 | DEBUG    | genvarloader._dataset._write:_read_variants_chunk:427 - maximum needed length 33298
2024-09-03 15:10:50.719 | DEBUG    | genvarloader._dataset._write:_read_variants_chunk:428 - minimum needed length 33038
2024-09-03 15:10:51.136 | INFO     | genvarloader._

  0%|          | 0/1 [00:00<?, ?it/s]

2024-09-03 15:10:51.752 | INFO     | genvarloader._dataset._write:write:170 - Finished writing.


Note that `gvl.write` will also automatically use the intersection of samples from source files. In this case, they are perfectly matched to each other. But, if we had used PLINK files for the full 3,202 samples from the 1000 Genomes Project then it would have identified and used the 451 intersecting samples.

## Inspecting the dataset

In [7]:
ds = gvl.Dataset.open(ds_path)

2024-09-03 15:10:51.837 | INFO     | genvarloader._dataset:_open:175 - 
GVL store geuvadis.chr22.gvl
Is subset: False
# of regions: 5
# of samples: 451
Original region length: 32,768
Max jitter: 128
Has genotypes: True
Has tracks: ['depth']
2024-09-03 15:10:51.838 | WARNING  | genvarloader._dataset:open:235 - Genotypes found but no reference genome provided. This is required to reconstruct haplotypes. No reference or haplotype sequences can be returned by this dataset instance.


If we don't provide a reference genome to a dataset that has genotypes, we will get an informative warning and the dataset will never provide haplotypes. Let's go ahead and specify a reference genome.

In [8]:
ds = gvl.Dataset.open(ds_path, reference=reference)

2024-09-03 15:10:51.851 | INFO     | genvarloader._dataset:_open:122 - Loading reference genome into memory. This typically has a modest memory footprint (a few GB) and greatly improves performance.
2024-09-03 15:10:53.545 | INFO     | genvarloader._dataset:_open:175 - 
GVL store geuvadis.chr22.gvl
Is subset: False
# of regions: 5
# of samples: 451
Original region length: 32,768
Max jitter: 128
Has genotypes: True
Has tracks: ['depth']


Now that a reference genome is provided, haplotypes can be returned. We also are given some summary information to get a sense of what is available in this dataset. Let's use the dataset to inspect a few sequences and tracks and seeing how we can adjust what is returned as well.

In [9]:
ds[0]

(array([[b'A', b'A', b'T', ..., b'C', b'T', b'G'],
        [b'T', b'A', b'T', ..., b'G', b'G', b'C']], dtype='|S1'),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

Indexing into a GVL dataset corresponding to the raveled indices, so the 0-th index is the data for the first region and sample.

Since this dataset has jitter enabled (the maximum amount by default), we will get different data each time we access it. We can disable jittering, but we will still get randomly shifted data for haplotypes that are longer than the output length due to indels.

We also are receiving both haplotypes and tracks from the dataset, and they have an additional dimension for ploidy.

In [10]:
[a.shape for a in ds[0]]

[(2, 32768), (2, 32768)]

We can disable returning haplotypes and return reference sequences instead, and now the ploidy dimension will be gone. We can also see that disabling jitter will increase the sequence length to the maximum available. We could disable jittering without altering sequence length by slicing the them  on-the-fly with a transform.

In [11]:
ref_ds = ds.with_settings(jitter=0, return_sequences='reference')
[a.shape for a in ref_ds[0]]

[(33024,), (33024,)]

We can also slice the dataset or use lists/arrays of indices to get batches of data.

In [12]:
[a.shape for a in ds[:10]]

[(10, 2, 32768), (10, 2, 32768)]

In [13]:
[a.shape for a in ds[[0, 3, 999]]]

[(3, 2, 32768), (3, 2, 32768)]

## Optional: pre-computing transformed tracks and saving them to disk

Suppose we would like to normalize the read depth across the dataset to account for library size. We could compute this on-the-fly, but GVL also offers a way to write this data back to disk to cache this computation and potentially improve performance. Note that this is the most technical part of this tutorial, so feel free to skip this and come back later.

In [14]:
sample_library_sizes = (
    pl.Series(ds.samples)
    .to_frame("sample")
    .join(bigwig_table, on="sample")["read_count"]
    .to_numpy()
)
sample_library_sizes[:5]

array([26025190, 26304702, 27759874, 23282559, 17936970])

For the transform, we'll use [`gvl.Dataset.write_transformed_track`](https://genvarloader.readthedocs.io/en/latest/api.html#genvarloader.Dataset.write_transformed_track) which expects a transform function to be given. From the docs:

> The arguments given to the transform will be the dataset indices, region indices, and sample indices as numpy arrays and the tracks themselves as a [`Ragged`](https://genvarloader.readthedocs.io/en/latest/api.html#genvarloader.Ragged) array with shape (regions, samples). The tracks must be a [`Ragged`](https://genvarloader.readthedocs.io/en/latest/api.html#genvarloader.Ragged) array since regions may be different lengths to accomodate indels. This function should then return the transformed tracks as a [`Ragged`](https://genvarloader.readthedocs.io/en/latest/api.html#genvarloader.Ragged) array with the same shape and lengths.

Below, you can see an example of a transform of ragged data that uses Numba to accelerate the computation. Note that working with Ragged data is generally not necessary with on-the-fly transformations, since the data is processed to be uniform length before any transformation.

In [15]:
@nb.njit(parallel=True, nogil=True, fastmath=True)
def inner_transform(s_idx, data, offsets):
    log_cpm = np.empty_like(data)
    for i in nb.prange(len(offsets) - 1):
        start = offsets[i]
        end = offsets[i + 1]
        sample = s_idx[i]
        log_cpm[start:end] = np.log1p(
            data[start:end] / sample_library_sizes[sample] * 1e6
        )
    return log_cpm


def log_cpm(ds_idx, r_idx, s_idx, tracks: gvl.Ragged[np.float32]):
    data = inner_transform(s_idx, tracks.data, tracks.offsets)
    return gvl.Ragged.from_offsets(data, tracks.shape, tracks.offsets)


ds = ds.write_transformed_track("lcpb", "depth", log_cpm, overwrite=True, max_mem=4 * 2**30)

  0%|          | 0/1 [00:00<?, ?it/s]

## On-the-fly transformations

One thing you may have noticed is that the sequences are output as ASCII characters. We'll often need to either tokenize or one-hot encode them for machine learning models. We can do this on-the-fly with, for example, fast implementations from [SeqPro](https://github.com/ML4GLand/SeqPro), but in general arbitrary transformations can be used.

In [16]:
def tokenize_transform(haplotypes, tracks):
    return sp.tokenize(haplotypes, dict(zip(sp.DNA.alphabet, range(4))), 4), tracks


def ohe_transform(haplotypes, tracks):
    return sp.DNA.ohe(haplotypes), tracks


token_ds = ds.with_settings(transform=tokenize_transform)
ohe_ds = ds.with_settings(transform=ohe_transform)

In [17]:
token_ds[0][0], ohe_ds[0][0]

(array([[1, 3, 0, ..., 1, 2, 1],
        [0, 3, 3, ..., 1, 2, 1]], dtype=int32),
 array([[[0, 0, 1, 0],
         [0, 0, 0, 1],
         [1, 0, 0, 0],
         ...,
         [0, 1, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 1, 0]],
 
        [[0, 0, 1, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 1],
         ...,
         [0, 0, 0, 1],
         [0, 1, 0, 0],
         [0, 1, 0, 0]]], dtype=uint8))

## Splitting datasets

Suppose we're training a model and thus need to split our dataset. Let's create a subset of the dataset to the first 400 samples for training.

In [18]:
train_ds = ds.subset_to(samples=ds.samples[:400])
train_ds

GVL store geuvadis.chr22.gvl
Is subset: True
# of regions: 5
# of samples: 400
Original region length: 32,768
Max jitter: 128
Has genotypes: True
Has tracks: ['depth', 'lcpb']

We can see that now the dataset is marked as a subset and the # of samples has reduced from 451 to 400. Some other properties reflect these changes as well:

In [19]:
len(ds), len(train_ds), ds.shape, train_ds.shape

(2255, 2000, (5, 451), (5, 400))

## Getting a PyTorch DataLoader

In [20]:
train_dl = train_ds.to_dataloader(batch_size=128, shuffle=True)
batch = next(iter(train_dl))
batch[0].shape, batch[1].shape

(torch.Size([128, 2, 32768]), torch.Size([128, 2, 32768]))

Note that since GVL provides a map-style PyTorch Dataset, this approach is compatible with distributed data parallel (DDP) for use across multiple GPUs or nodes.